In [4]:
# Librarys
%run ForeSignalScraper.ipynb function(Login)
%run ForeSignalScraper.ipynb function(CustomizeFoSignals)
%run TradingViewScaper.ipynb function(Price)
%run OpenOrdersMT5.ipynb function(Login_mt5,Expiration_time,Determination_Set_Order)
from datetime import datetime
from collections import OrderedDict
import pandas as pd
import random
import time

In [5]:
# Currencys list and Desired Time to Refresh wbsites
Currencys_list = ['EURUSD','USDCHF','GBPUSD','USDJPY','USDCAD','AUDUSD','EURJPY','NZDUSD','GBPCHF']
desired_time = int(input('Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : '))

# Foresignal login Information
FoLogin_url = 'https://foresignal.com/en/login/index'
Fouser_name = 'xabiz'
FoUser_password = 'tohid0010011'

# Foresignal Scrape Specification
FoWebsite_url = 'https://foresignal.com/en/'
Foclassorid = 'class'
Foclassoridvalue = 'd-flex flex-row flex-wrap signal-row'

#Login to Foresignal and Scrape Data
ForeSignalDriver = Login(FoLogin_url,Fouser_name,FoUser_password)

#Login to Tradingview and Scrape Data
final_list = Price(2,Currencys_list)

#MetaTrader5 Login Informations and login to Software
login_id = 3060009
password = '$9Bj#5qL'
server = 'FIBOGroup-MT5 Server'
Login_mt5 (login_id,password,server)

# Empty Dicts and Lists
final_informations = OrderedDict()
FoFinalList = list()
Reference_list = list()

Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : 10


In [6]:

while True:

    FoFinalList_Start_len = len(FoFinalList)
    
    for items in CustomizeFoSignals(ForeSignalDriver,FoWebsite_url,Foclassorid,Foclassoridvalue):
        if len(items) > 10 and items not in FoFinalList:
            FoFinalList.append(items)
#     print(FoFinalList)
    
    FoFinalList_Final_len = len(FoFinalList)

    if FoFinalList_Start_len < FoFinalList_Final_len:
        
        Tradingview_price_dictionary = OrderedDict()

        for item in range (0,9):
            now = datetime.now()
            final_list[0].switch_to.window(final_list[0].window_handles[item])
            current_date = f'{Currencys_list[item]} : {now}'
            current_price = final_list[1][item].text
            
            Tradingview_price_dictionary[current_date] = current_price
            
#         print(Tradingview_price_dictionary)
        
        for item_fo in FoFinalList:
            for k,v in Tradingview_price_dictionary.items():
                if item_fo[0] in k[:6]:
                    item_fo.append(k)
                    item_fo.append(v)
                    final_informations[f'{item_fo[:11]}'] = item_fo
        
        Reference_list_start_len = len(Reference_list)

        Reference_list = list(final_informations.items()) 
        
        Reference_list_final_len = len(Reference_list)

        if Reference_list_start_len < Reference_list_final_len:
            
            for items in Reference_list[Reference_list_start_len:Reference_list_final_len]:
            
                print(items[1])
                Symbole = items[1][0]
                Entry_Price = float(items[1][6])
                Stoploss_Price = float(items[1][10])
                Takeprofit_Price = float(items[1][8])
                Current_Price = float(items[1][12])
                current_time = items[1][4].replace(':',' ').split()
                expiration = Expiration_time (int(current_time[0]),int(current_time[1]))
                                       
                if items[1][5] == 'Buy at' and Entry_Price > Current_Price:
                    Type_of_Trade = 'buy stop'
                
                elif items[1][5] == 'Buy at' and Entry_Price < Current_Price:
                    Type_of_Trade = 'buy limit'
            
                elif items[1][5] == 'Sell at' and Entry_Price < Current_Price:
                    Type_of_Trade = 'sell stop'
            
                elif items[1][5] == 'Sell at' and Entry_Price > Current_Price:
                    Type_of_Trade = 'sell limit'
                
                request = Determination_Set_Order(Type_of_Trade, Symbole , 0.05 ,Entry_Price
                                              ,Stoploss_Price,Takeprofit_Price,expiration)   
                result = mt.order_send(request)
                result
                        
    delay_time = random.randrange(0,desired_time)
    time.sleep(delay_time)


['USDCHF', 'From UTC-08:00', '09:13', 'Till UTC-08:00', '00:13', 'Buy at', '0.8811', 'Take profit* at', '0.883', 'Stop loss at', '0.8788', 'USDCHF : 2023-11-27 22:27:41.447222', '0.88039']
['USDCAD', 'From UTC-08:00', '11:06', 'Till UTC-08:00', '20:06', 'Buy at', '1.3623', 'Take profit* at', '1.3637', 'Stop loss at', '1.3606', 'USDCAD : 2023-11-27 22:27:41.674160', '1.36248']
['AUDUSD', 'From UTC-08:00', '08:49', 'Till UTC-08:00', '16:49', 'Sell at', '0.6595', 'Take profit* at', '0.658', 'Stop loss at', '0.6614', 'AUDUSD : 2023-11-27 22:27:41.748509', '0.66054']
['EURJPY', 'From UTC-08:00', '12:19', 'Till UTC-08:00', '18:19', 'Buy at', '162.75', 'Take profit* at', '162.9', 'Stop loss at', '162.57', 'EURJPY : 2023-11-27 22:27:41.817524', '162.809']
['NZDUSD', 'From UTC-08:00', '11:46', 'Till UTC-08:00', '20:46', 'Sell at', '0.6108', 'Take profit* at', '0.6094', 'Stop loss at', '0.6125', 'NZDUSD : 2023-11-27 22:27:42.047430', '0.61002']
['GBPCHF', 'From UTC-08:00', '10:30', 'Till UTC-08:

KeyboardInterrupt: 